<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/10_pruning_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISION_ARTIFICIAL - Notebook 10: Pruning y Quantization

🔹 Importar librerías

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow_model_optimization.sparsity import keras as sparsity
import numpy as np
import os

🔹 Cargar datos procesados

In [ ]:
processed_dir = "../data/processed/"
x_train = np.load(os.path.join(processed_dir, "x_train.npy"))
y_train = np.load(os.path.join(processed_dir, "y_train.npy"))
x_test  = np.load(os.path.join(processed_dir, "x_test.npy"))
y_test  = np.load(os.path.join(processed_dir, "y_test.npy"))


🔹 Cargar modelo original

In [ ]:
model_path = "../models/cnn_best.keras"
model = keras.models.load_model(model_path)
model.summary()


🔹 Configurar pruning

In [ ]:
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                 final_sparsity=0.5,
                                                 begin_step=0,
                                                 end_step=np.ceil(len(x_train)/32).astype(np.int32)*10)
}

pruned_model = sparsity.prune_low_magnitude(model, **pruning_params)

🔹 Compilar y entrenar modelo podado

In [ ]:
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [sparsity.UpdatePruningStep()]

pruned_model.fit(x_train, y_train,
                 batch_size=32,
                 epochs=10,
                 validation_data=(x_test, y_test),
                 callbacks=callbacks)

🔹 Strip pruning y guardar modelo podado

In [ ]:
final_model = sparsity.strip_pruning(pruned_model)
pruned_model_path = "../models/cnn_pruned.keras"
final_model.save(pruned_model_path)
print(f"✅ Modelo podado guardado en '{pruned_model_path}'")

🔹 Quantization: convertir a TensorFlow Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_model_path = "../models/cnn_pruned.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ Modelo quantizado guardado en '{tflite_model_path}'")

NameError: name 'tf' is not defined